In [0]:
import numpy as np
import pandas as pd
from itertools import combinations 

In [0]:
!wget -q https://s3.eu-central-1.amazonaws.com/meetupsocialnetwork/group_nw_.csv
  
!wget -q https://s3.eu-central-1.amazonaws.com/meetupsocialnetwork/groups_nodefile.csv

In [0]:
group_edge = pd.read_csv("group_nw_.csv")
group_node = pd.read_csv("groups_nodefile.csv")

print(group_node.head())
print(group_edge.head())

   NodeId                      GroupName  ...         Date Created  Rating
0    6388         Alternative Health NYC  ...  2002-11-21 16:50:46    4.39
1    6510      Alternative Energy Meetup  ...  2003-05-20 14:48:54    4.31
2    8458              NYC Animal Rights  ...  2004-03-27 09:55:41    4.84
3    8940  The New York City Anime Group  ...  2002-11-16 04:49:16    4.46
4   10104             NYC Pit Bull Group  ...  2003-10-22 21:39:49    4.09

[5 rows x 7 columns]
   Unnamed: 0  ...                                       comm_members
0           0  ...  [3, 1418715, 1671282, 2148610, 2319427, 270929...
1           1  ...  [3, 1418715, 1584644, 2148610, 2709298, 330086...
2           2  ...  [3, 1418715, 1597206, 1671282, 2148610, 222025...
3           3  ...  [3, 1418715, 1597206, 1671282, 2148610, 249616...
4           4  ...  [3, 1584644, 1597206, 2684823, 2741328, 318142...

[5 rows x 6 columns]


In [0]:
group_node.head()


,NodeId,GroupName,CategoryID,# Members,City,Date Created,Rating
0,6388,Alternative Health NYC,14,1440,New York,2002-11-21 16:50:46,4.39
1,6510,Alternative Energy Meetup,4,969,New York,2003-05-20 14:48:54,4.31
2,8458,NYC Animal Rights,26,2930,New York,2004-03-27 09:55:41,4.84
3,8940,The New York City Anime Group,29,5080,New York,2002-11-16 04:49:16,4.46
4,10104,NYC Pit Bull Group,26,2097,New York,2003-10-22 21:39:49,4.09


In [0]:
group_edge.head()

,Unnamed: 0,EdgeID,group_from,group_to,weight,comm_members
0,0,0,490552,1474611,57,"[3, 1418715, 1671282, 2148610, 2319427, 270929..."
1,1,1,490552,1490492,66,"[3, 1418715, 1584644, 2148610, 2709298, 330086..."
2,2,2,490552,1515830,167,"[3, 1418715, 1597206, 1671282, 2148610, 222025..."
3,3,3,490552,1574965,46,"[3, 1418715, 1597206, 1671282, 2148610, 249616..."
4,4,4,490552,1669000,62,"[3, 1584644, 1597206, 2684823, 2741328, 318142..."


In [0]:
l = [2,4,5,6,7,8,9]

ll = list(combinations(l, 3))
print(ll)

[(2, 4, 5), (2, 4, 6), (2, 4, 7), (2, 4, 8), (2, 4, 9), (2, 5, 6), (2, 5, 7), (2, 5, 8), (2, 5, 9), (2, 6, 7), (2, 6, 8), (2, 6, 9), (2, 7, 8), (2, 7, 9), (2, 8, 9), (4, 5, 6), (4, 5, 7), (4, 5, 8), (4, 5, 9), (4, 6, 7), (4, 6, 8), (4, 6, 9), (4, 7, 8), (4, 7, 9), (4, 8, 9), (5, 6, 7), (5, 6, 8), (5, 6, 9), (5, 7, 8), (5, 7, 9), (5, 8, 9), (6, 7, 8), (6, 7, 9), (6, 8, 9), (7, 8, 9)]


In [0]:
beta = 0.3
theta = 0.6
nodes = group_node["NodeId"]
print(nodes[0])

print(group_edge.shape[0])
group_edge.drop(group_edge.loc[group_edge['weight'] == 1].index, inplace=True)
group_edge.drop(group_edge.loc[group_edge['weight'] == 2].index, inplace=True)
group_edge.drop(group_edge.loc[group_edge['weight'] == 3].index, inplace=True)
group_edge.drop(group_edge.loc[group_edge['weight'] == 4].index, inplace=True)
print(group_edge.shape[0])
#temp = group_edge.loc[group_edge['group_from'] == nodes[0]]
#print(group_edge.loc[group_edge['group_from'] == nodes[0]].shape[0])
H = {}
node_pairs = list(combinations(nodes, 2))
RoleSim = {}
for pair in node_pairs:
  RoleSim[pair] = 1
  
  
def calc_weight(u, v):
  neigh1 = list(group_edge.loc[group_edge['group_from'] == u]['group_to'])
  neigh2 = list(group_edge.loc[group_edge['group_from'] == v]['group_to'])
  max_weight = 0
  n = len(neigh1)
  v_trips = list(combinations(neigh2, n))
  del neigh2
  for triples in v_trips:
    for n1 in neigh1:
      temp = RoleSim[(n1, triples[0])] + RoleSim[(n1, triples[1])] + RoleSim[(n1, triples[2])]
      if temp > max_weight:
        max_weight = temp
  return max_weight, n
  
  
degrees = {}
for n in nodes:
  temp = group_edge.loc[group_edge['group_from'] == n]
  degrees[n] = []
  degrees[n].append(temp.shape[0])
  temp1 = []
  for neigh in list(temp['group_to']):
    if neigh in degrees:
      temp1.append(degrees[neigh][0])
    else:
      temp1.append(group_edge.loc[group_edge['group_from'] == neigh].shape[0])
  temp1.sort()
  degrees[n].append(temp1)
  del temp1
  del temp

for n1 in nodes:
  for n2 in nodes:
    if n1 == n2:
      continue
    if (theta * degrees[n1][0] < degrees[n2][0] < degrees[n1][0]):
      m11 = (1 - beta) * (min(degrees[n1][1][0])/max(degrees[n2][1][0])) + beta
      if (degrees[n2][1][0] <= degrees[n1][1][0]) and (degrees[n1][0] -1 + m11 < theta * degrees[n2][0]):
        continue
      if n1 < n2:
        w = calc_weight(n1, n2)
      else:
        w = calc_weight(n2, n1)
      if (w >= theta * degrees[n1][0]):
        H[(n1, n2)] = ((1 - beta) * w )/degrees[n1][0] + beta
      
      
def calc_rolesim(nodes_pair, beta):
  for pair in nodes_pair:
    
    if pair[0] < pair[1]: 
      wt, n = calc_weight(pair[0], pair[1])
      RoleSim[pair] = ((1 - beta) * (wt / (degrees[pair[0]]) + degrees[pair[1]] - n)) + beta
      
    else:
      wt, n = calc_weight(pair[1], pair[0])
      RoleSim[pair] = ((1 - beta) * (wt / (degrees[pair[0]]) + degrees[pair[1]] - n)) + beta

6388
10969547
3105083


NameError: ignored